In [78]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import pandas as pd
import numpy as np
from functions import get_data, make_model
import pickle

instrument = "EUR_USD"
params = {
    "from": "2019-01-01T00:00:00Z",
    "to": "2021-01-29T00:00:00Z",
    "granularity": "D",
    "count": 5000,
}
directory = 'training/v2/_4/'
client = oandapyV20.API(access_token='USE_OWN_ACCESS_TOKEN')


In [3]:
get_data(instrument, params, "csv/daily_candles.csv", client)

In [4]:
params['from'] = "2019-10-07T16:00:00Z"
params['granularity'] = 'M15'
get_data(instrument, params, 'csv/five_min_candles.csv', client)

In [5]:
def cci(df, period=20):
    df['TP'] = (df['high']+df['low']+df['close'])/3
    df['TP_SMA']=0.0
    df['CCI']=0.0
    for i in range(period, len(df)):
        df.loc[i, 'TP_SMA'] = np.mean(df.loc[i-period:i, 'TP'])
        mean_deviation = np.mean(np.abs(df.loc[i, 'TP_SMA'] - df.loc[i-period:i, 'TP']))
        df.loc[i, 'CCI'] = (df.loc[i, 'TP'] - df.loc[i, 'TP_SMA'])/(1.5 * mean_deviation)

def stochastic(df, period=14): # same as %K
    df['%K'] = 0.0
    for i in range(period, len(df)):
        high = np.max(df.loc[i-period:i, 'high'])
        low = np.min(df.loc[i-period:i, 'low'])
        close = df.loc[i, 'close']
        df.loc[i, '%K'] = (close-low)/(high-low)
def rsi(df, period=14):
    df['RSI'], df['mean_gain'], df['mean_loss'] = 0.0, 0.0, 0.0
    gain, loss = 0, 0
    for j in range(0, period):
        change = df.loc[j+1, 'close']/df.loc[j, 'close']
        if change>=1:
            gain += change-1
        else:
            loss += 1-change
    df.loc[period, 'mean_gain'] = gain/period
    df.loc[period, 'mean_loss'] = loss/period
    RS = gain/loss
    df.loc[period, 'RSI'] = 1 - 1/(1+RS)
    for i in range(period+1, len(df)):
        gain, loss = 0, 0
        change = df.loc[i, 'close']/df.loc[i-1, 'close']
        if change>=1:
            gain += change-1
        else:
            loss += 1-change
        df.loc[i, 'mean_gain'] = (df.loc[i-1, 'mean_gain']*13 + gain)/period
        df.loc[i, 'mean_loss'] = (df.loc[i-1, 'mean_loss']*13 + loss)/period
        RS = df.loc[i, 'mean_gain']/df.loc[i, 'mean_loss']
        df.loc[i, 'RSI'] = 1 - 1/(1+RS)
def ema(df, period):
    s = 'ema-{}'.format(period)
    df[s]=0.0
    df.at[period, s] = np.mean(df.loc[:period, 'close'])
    multiplier = 2/(1+period)
    for i in range(period+1, len(df)):
        df.at[i, s] = df.at[i, 'close']*multiplier + df.at[i-1, s]*(1-multiplier)

def atr(df, period=6):
    df['ATR']=0.0
    df['TR']=0.0
    for i in range(1, len(df)):
        h, l, c = df.loc[i, 'high'], df.loc[i, 'low'], df.loc[i-1, 'close']
        df.loc[i, 'TR'] = np.max([h-l, abs(h-c), abs(l-c)])
    for i in range(period, len(df)):
        df.loc[i, 'ATR'] = np.mean(df.loc[i-period+1:i+1, 'TR'])
def add_indicators(df):
    cci(df)
    stochastic(df)
    rsi(df)
    ema(df, 5)
    ema(df, 10)
    ema(df, 20)
    atr(df)

In [6]:
daily = pd.read_csv("csv/daily_candles.csv")
fifteen = pd.read_csv('csv/five_min_candles.csv')

daily = daily.drop(['time'], axis=1)
fifteen = fifteen.drop(['time'], axis=1)

daily = daily.astype(np.float64)
fifteen = fifteen.astype(np.float64)
add_indicators(daily), add_indicators(fifteen)

(None, None)

In [7]:
price_scaling = ['open', 'high', 'low', 'close', 'TP', 'TP_SMA', 'ema-5', 'ema-10', 'ema-20']
thousand_scaling = ['range', 'change', 'ATR']
training_features = ['open', 'high', 'low', 'close', 'volume', 'range', 'change', 'TP', 'TP_SMA', 'CCI', '%K', 'RSI', 'ema-5', 'ema-10', 'ema-20', 'ATR']

daily = daily.loc[20:, training_features]
fifteen = fifteen.loc[20:, training_features]
daily[price_scaling] = (daily[price_scaling]-1.15)*5
fifteen[price_scaling] = (fifteen[price_scaling]-1.15)*5
fifteen[thousand_scaling] *= 10e3
daily[thousand_scaling] *= 10e3
daily[['range', 'change']] /= 10
daily['volume'] /= 10e5
fifteen['volume'] /= 10e3
# daily[15:25]

In [8]:
# from sklearn.preprocessing import MinMaxScaler
# daily = np.array(daily)
# fifteen = np.array(fifteen)
# scaler = MinMaxScaler().fit(np.concatenate((daily[:, 4:], five[:, 4:]), axis=0))
# daily[:, 4:] = scaler.transform(daily[:, 4:])
# five[:, 4:] = scaler.transform(five[:, 4:])
# pickle.dump(scaler, open((directory+'scaler.p'), 'wb'))
# print(directory+'scaler.p')

In [9]:
daily = np.array(daily)
fifteen = np.array(fifteen)
lb_five = 288
lb_days = 180
z=179
future_candles = 40
loss, gain = 10, 30
x1, x2, y = [], [], []
for i in range(len(fifteen)-lb_five-future_candles):
    if i%96==0:
        z+=1
    x1.append(daily[z-lb_days:z])
    x2.append(fifteen[i:i+lb_five])
    close = fifteen[i+lb_five-1][3]
    up = True
    down = True
    for j in range(future_candles):
        c = fifteen[i+lb_five+j]
        if down and c[1]-close>=0.0001*loss*5:
            down = False
        elif up and c[2]-close<=-0.0001*loss*5:
            up = False
        if c[1]-close>=0.0001*gain*5 and up:
            y.append(np.array([0, 0, 1]))
            break
        if c[2]-close<=-0.0001*gain*5 and down:
            y.append(np.array([1, 0, 0]))
            break
        if not down and not up:
            y.append(np.array([0, 1, 0]))
            break
    if len(y)!=i+1:
        y.append(np.array([0, 1, 0]))
x1=np.array(x1).astype(np.float64)
x2=np.array(x2).astype(np.float64)
y=np.array(y).astype(np.float64)

In [64]:
from sklearn.model_selection import train_test_split
trainXd, testXd, trainXf, testXf, trainY, testY = train_test_split(x1, x2, y, test_size=0.2, shuffle=False)

In [79]:
import tensorflow as tf
# import keras as k
checkpoint_path = directory+'cp.ckpt'
# cp_callback = k.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

In [12]:
def eval_metric(realy, predy, cutoff=0.2):
    ans = []
    total_times_invested=0
    short=0
    for y in predy:
        if y[0] >= cutoff:
            ans.append([1, 0, 0])
            total_times_invested+=1
            short+=1
        elif y[2] >= cutoff:
            ans.append([0, 0, 1])
            total_times_invested+=1
        else:
            ans.append([0, 1, 0])
    ans = np.array(ans)

    correctly_invested=0
    for i in range(len(ans)):
        if np.array_equal(ans[i], realy[i]) and (np.array_equal(ans[i], [1, 0, 0]) or np.array_equal(ans[i], [0, 0, 1])):
            correctly_invested+=1
    if total_times_invested==0:
        return 0
    # print(short)
    return [correctly_invested, total_times_invested, float(correctly_invested)/total_times_invested]

In [42]:
class TradeMetric(tf.keras.metrics.Metric):
    def __init__(self, name='trade_acc', **kwargs):
        super().__init__(name=name, **kwargs)
        self.good_trades = self.add_weight(name='gt', initializer='zeros', dtype=tf.float32)
        self.total_trades = self.add_weight(name='tt', initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        pred_short = tf.where(y_pred[:, 0]>0.5, 1.0, 0.0)
        pred_long = tf.where(y_pred[:, 2]>0.5, 1.0, 0.0)
        real_short = tf.where(y_true[:, 0]==1, 1.0, 0.0)
        real_long = tf.where(y_true[:, 2]==1, 1.0, 0.0)
        self.total_trades.assign_add(tf.reduce_sum(pred_short))
        self.total_trades.assign_add(tf.reduce_sum(pred_long))
        s1 = tf.reduce_sum(tf.cast(tf.equal(real_long, pred_long), dtype=tf.float32))
        s2 = tf.reduce_sum(tf.cast(tf.equal(real_short, pred_short), dtype=tf.float32))
        self.good_trades.assign_add(s1)
        self.good_trades.assign_add(s2)

    def result(self):
        return tf.realdiv(self.good_trades, self.total_trades)
        # return self.total_trades

    def reset_states(self):
        self.good_trades.assign(0.0)
        self.total_trades.assign(0.0)
class TimesTraded(TradeMetric):
    def __init__(self, name='times_traded', **kwargs):
        super().__init__(name=name, **kwargs)
        self.good_trades = self.add_weight(name='gt', initializer='zeros', dtype=tf.float32)
        self.total_trades = self.add_weight(name='tt', initializer='zeros', dtype=tf.float32)
    def result(self):
        return self.total_trades

In [65]:
import keras
import tensorflow as tf
import tensorflow_addons as tfa
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras import optimizers
def model_init():
    daily_lstm_input = Input(shape=(180, 16), name='daily_input')
    x = LSTM(180, name='daily_lstm')(daily_lstm_input)
    x = Dropout(0.2, name='daily_dropout')(x)
    daily_branch = Model(inputs=daily_lstm_input, outputs=x)

    fifteen_lstm_input = Input(shape=(288, 16), name='M15_input')
    y = LSTM(288, name='M15_lstm')(fifteen_lstm_input)
    y = Dropout(0.2, name='M15_dropout')(y)
    fifteen_branch = Model(inputs=fifteen_lstm_input, outputs=y)

    combined = concatenate([daily_branch.output, fifteen_branch.output])
    z = Dense(128, activation="sigmoid")(combined)
    z = Dropout(0.2)(z)
    z = Dense(3, activation="softmax")(z)

    model = Model(inputs=[daily_branch.input, fifteen_branch.input], outputs=z)
    adam = optimizers.Adam(lr=0.0005)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['categorical_accuracy'], run_eagerly=False)
    return model

In [53]:
# earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [66]:
model=model_init()
model.fit(x=[trainXd, trainXf], y=trainY, batch_size=200, epochs=10, validation_data=([testXd, testXf], testY), callbacks=None)

Epoch 1/10
129/129 [==============================] - 238s 2s/step - loss: 0.8979 - categorical_accuracy: 0.6385 - val_loss: 0.9379 - val_categorical_accuracy: 0.6169

In [75]:
model.fit(x=[trainXd, trainXf], y=trainY, batch_size=200, epochs=1, validation_data=([testXd, testXf], testY), callbacks=None)

129/129 [==============================] - 206s 2s/step - loss: 0.6045 - categorical_accuracy: 0.7350 - val_loss: 1.2214 - val_categorical_accuracy: 0.5714

In [80]:
model.save_weights(checkpoint_path)

In [84]:
model.load_weights(checkpoint_path).expect_partial()

In [76]:
import time
start=time.perf_counter()
model.call = tf.function(model.call, experimental_relax_shapes=True)
# model(..., training=False)
pY = model([testXd, testXf], training=False)
print(time.perf_counter()-start)

17.883878142001777


In [77]:
print(np.sum(testY, axis=0))
print(eval_metric(testY, pY, 0.5))

[1141. 3972. 1326.]
[245, 960, 0.2552083333333333]
